In [43]:
import numpy as np
import matplotlib.pyplot as plt

class Neuralnet:
    def __init__(self, neurons):
        self.layers = len(neurons)

        # Learning rate
        self.rate = .01

        # Input vectors
        self.inputs = []
        # Output vectors
        self.outputs = []
        # Error vectors
        self.errors = []
        # Weight matrices
        self.weights = []
        # Bias vectors
        self.biases = []

        for layer in range(self.layers):
            # Create the input, output, and error vector
            self.inputs.append(np.empty(neurons[layer]))
            self.outputs.append(np.empty(neurons[layer]))
            self.errors.append(np.empty(neurons[layer]))

        for layer in range(self.layers - 1):
            # Create the weight matrix
            self.weights.append(np.random.normal(
                scale=1.0/np.sqrt(neurons[layer]),
                size=[neurons[layer], neurons[layer + 1]]
            ))
            # Create the bias vector
            self.biases.append(np.random.normal(
                scale=1.0/np.sqrt(neurons[layer]),
                size=neurons[layer + 1]
            ))

    def feedforward(self, inputs):
        # Set input neuron inputs
        self.inputs[0] = inputs
        for layer in range(self.layers - 1):
            # Find output of this layer from its input
            self.outputs[layer] = np.tanh(self.inputs[layer])
            # Find input of next layer from output of this layer and weight matrix (plus bias)
            self.inputs[layer + 1] = np.dot(self.weights[layer].T, self.outputs[layer]) + self.biases[layer]
        self.outputs[-1] = np.tanh(self.inputs[-1])

    def backpropagate(self, targets):
        # Calculate error at output layer
        self.errors[-1] = self.outputs[-1] - targets
        # Calculate error vector for each layer
        for layer in reversed(range(self.layers - 1)):
            gradient = 1 - self.outputs[layer] * self.outputs[layer]
            self.errors[layer] = gradient * np.dot(self.weights[layer], self.errors[layer + 1])
        # Adjust weight matrices and bias vectors
        for layer in range(self.layers - 1):
            self.weights[layer] -= self.rate * np.outer(self.outputs[layer], self.errors[layer + 1])
            self.biases[layer] -= self.rate * self.errors[layer + 1]

# Create a neural network that accepts a 28 by 28 array as input and has 10 output neurons
net = Neuralnet([28 * 28, 200, 10])

# Extract handwritten digits from files
data = pd.read_csv("../train.csv") #Adjust filepath
y_label = data["label"]
data = data.drop("label",axis=1)

# Train neural network on entire data set multiple times
for epoch in range(10):
    # Total error for this epoch
    error = 0
    # Choose a sample index
    for sample in range(len(data)):
        # Feed input data to neural network
        net.feedforward(data.iloc[sample])
        # Target output consists of -1s except for matching digit
        targets = np.full(10, -1, dtype=np.float32)
        targets[y_label[sample]] = 1
        # Train neural network based on target output
        net.backpropagate(targets)
        error += np.sum(net.errors[-1] * net.errors[-1])
    print('Epoch ' + str(epoch) + ' error: ' + str(error))

test = pd.read_csv("/Users/matthewzhou/Desktop/Image Recognition/test.csv")
predictions = ["Label"]
for i in range(len(test)):
    net.feedforward(test.iloc[i])
    predictions.append(np.argmax(net.outputs[-1]))
df = pd.DataFrame(predictions)
df.to_csv("results.csv", index_label="ImageID")


Epoch 0 error: 27217.6420544
Epoch 1 error: 15366.2687653
Epoch 2 error: 12049.3146305
Epoch 3 error: 10171.1956274
Epoch 4 error: 9035.98153577
Epoch 5 error: 7755.60143441
Epoch 6 error: 6977.1782606
Epoch 7 error: 6071.49979587
Epoch 8 error: 5166.32366389
Epoch 9 error: 5219.68792452
